In [ ]:
import pulp
import math

class_to_int_mapper = {
    'price': {
        "1400": 4,
        "1200": 3,
        "1000": 2,
        "800": 1
    },
    'certification': {
        "None": 4,
        "Basic": 3,
        "3rd-party": 2,
        "Full": 1
    },
    'payment': {
        "Full": 4,
        "1M": 3,
        "3M": 2,
        "6M": 1
    }
}

reversed_class_to_int_mapper = {
    'price': {v: k for k, v in class_to_int_mapper['price'].items()},
    'certification': {v: k for k, v in class_to_int_mapper['certification'].items()},
    'payment': {v: k for k, v in class_to_int_mapper['payment'].items()}
}

# Function to perform the actual Linear Programming for a single set of parameters
def solve_lp(max_point, lambda_factor, agents_value, partner_value, epsilon=0.0001):
    """
    Solves a Linear Programming problem for the given parameters.

    Args:
        max_point (int): Maximum points the agent can get.
        lambda_factor (float): Lambda factor balancing both parties' objectives.
        agents_value (dict): Agent's values for food, water, and firewood.
        partner_value (dict): Partner's values for food, water, and firewood.
        epsilon (float): Small adjustment to avoid floating point errors.

    Returns:
        tuple: A tuple containing the calculated scores and item allocations.
    """

    A_Val_price = agents_value['price'] # 3
    A_Val_certification = agents_value['certification'] #2
    A_Val_payment = agents_value['payment'] # 1

    # Agent's values adjusted to avoid floating point errors
    B_Val_price = partner_value['price'] - epsilon
    B_Val_certification = partner_value['certification'] - epsilon
    B_Val_payment = partner_value['payment'] - epsilon

    # Define the LP problem
    problem = pulp.LpProblem("Negotiation_Strategy_Max_Points", pulp.LpMaximize)

    # Define variables
    X = pulp.LpVariable("X", 1, 4, cat='Integer')  # price you get
    Y = pulp.LpVariable("Y", 1, 4, cat='Integer')  # certification you get
    Z = pulp.LpVariable("Z", 1, 4, cat='Integer')  # payment

    # Parameters
    B_threshold = 4  # Minimum points partner must get
    A_threshold = 4  # Minimum points agent must get

    # Objective function
    objective = (
        (A_Val_price * X + A_Val_certification * Y + A_Val_payment * Z) +
        (1 - lambda_factor) * ((B_Val_price * (5 - X) + B_Val_certification * (5 - Y) + B_Val_payment * (5 - Z)))
    )
    problem += objective

    # Constraints
    problem += A_Val_price * X + A_Val_certification * Y + A_Val_payment * Z <= max_point, "MaxPointsYouGet"
    problem += A_Val_price * X + A_Val_certification * Y + A_Val_payment * Z >= A_threshold, "AgentThreshold"
    problem += B_Val_price * (5 - X) + B_Val_certification * (5 - Y) + B_Val_payment * (5 - Z) >= B_threshold, "PartnerThreshold"

    # Solve the problem
    solver = pulp.PULP_CBC_CMD(msg=False, timeLimit=30, gapRel=0.01)
    problem.solve(solver)

    # Calculate scores
    A_score = A_Val_price * X.varValue + A_Val_certification * Y.varValue + A_Val_payment * Z.varValue
    B_score = B_Val_price * (5 - X.varValue) + B_Val_certification * (5 - Y.varValue) + B_Val_payment * (5 - Z.varValue)

    # Return results as a tuple
    return math.ceil(A_score), int(X.varValue), int(Y.varValue), int(Z.varValue)

# Function to loop over possible parameters and execute LP
def calculateBestOfferFromLP(maximum_value, lambda_value, agents_value, partner_value):
    """
    Iterates over parameters and calls the LP solver to find the best offers.

    Args:
        maximum_value (int): Maximum allowed value for the agent.
        lambda_value (float): Lambda value for balancing the objective.
        agents_value (dict): Agent's values for food, water, and firewood.
        partner_value (dict): Partner's values for food, water, and firewood.

    Returns:
        set: A set of tuples containing calculated scores and allocations.
    """
    set_of_offer = set()
    search_lower_bound = 5
    epsilon = 0.0001

    # Validate lambda value
    assert lambda_value is None or 0 <= lambda_value <= 1, "Lambda value should be between 0 and 1"

    # Create lambda lists for balancing
    lambda_lists = create_capped_window(lambda_value, window_size=0.3) if lambda_value else create_capped_window(0.5, window_size=0.5)

    # Loop through max values and lambda factors
    for mx in list(range(maximum_value, search_lower_bound, -1)):
        for l in lambda_lists:
            result = solve_lp(mx, l, agents_value, partner_value, epsilon)
            set_of_offer.add(result)

    return set_of_offer


In [ ]:

maximum_val = 24
lambda_val = 0.3
agents_preference_value =  {'price': 3, 'certification': 2, 'payment': 1}
partner_preference_value = {'price': 3, 'certification': 2, 'payment': 1}
solve_lp(maximum_val, lambda_val, agents_preference_value, partner_preference_value, epsilon=0.0001)
total_score, price_alloc, cert_alloc, pay_alloc = solve_lp(maximum_val, lambda_val, agents_preference_value, partner_preference_value, epsilon=0.0001)

print("Total Score:", total_score)
print("Price Allocation:", reversed_class_to_int_mapper['price'][price_alloc])
print("Certification Allocation:", reversed_class_to_int_mapper['certification'][cert_alloc])
print("Payment Allocation:", reversed_class_to_int_mapper['payment'][pay_alloc])

    #partner_value = {'food': 5, 'water': 4, 'firewood': 3}

    # lp_results1=calculateBestOfferFromLP(maximum_val, lambda_val, agents_preference_value, partner_preference_value)
    # print(sorted(lp_results1, key=lambda x: x[0], reverse=True))


Total Score: 19
Price Allocation: 1200
Certification Allocation: Basic
Payment Allocation: Full
